In [1]:
# Import Packages
import numpy as np
import os
import ast
import re

In [2]:
# Inital Data
initial_data_X = []
initial_data_y = []

for function_no in range(1,9):
    # Path relative to notebook location
    inital_file_path_X = f"../../data/initial_data/function_{function_no}/initial_inputs.npy"
    inital_file_path_y = f"../../data/initial_data/function_{function_no}/initial_outputs.npy"

    X = np.load(inital_file_path_X)
    initial_data_X.append(X)
    
    y = np.load(inital_file_path_y)
    initial_data_y.append(y)

In [3]:
# Weekly Submission Inputs
submission_file_path_X = f"../../data/submission_data/inputs.txt"

with open(submission_file_path_X, "r") as f:
    text = f.read()

cleaned = text.replace("\n", " ")
cleaned = re.sub(r'array\(', '(', cleaned)
            
blocks = re.split(r'\]\s*\[', cleaned)

parsed_blocks = []

for i, block in enumerate(blocks):
    # Fix the edges: add missing brackets removed by split
    if i == 0:
        block = block + "]"  # first block, add closing bracket
    elif i == len(blocks) - 1:
        block = "[" + block  # last block, add opening bracket
    else:
        block = "[" + block + "]"  # middle blocks, add both

    # Replace "([ ... ])" → "[ ... ]" to make valid Python
    block = re.sub(r'\(\[', '[', block)
    block = re.sub(r'\]\)', ']', block)

    # Evaluate
    parsed_list = ast.literal_eval(block)
    
    # Create List
    parsed_blocks.append(parsed_list)

submission_data_X = []
X = []

for i in range (0,8):
    for block in parsed_blocks:
        X.append(block[i])
    submission_data_X.append(X)
    X = []

In [4]:
# Weekly Submission Outputs
submission_file_path_y = f"../../data/submission_data/outputs.txt"
data = []

with open(submission_file_path_y, "r") as f:
    for line in f:
        line = line.strip()
        line = line.replace("np.float64", "")
        values = ast.literal_eval(line)
        data.append(values)

data = np.array(data, dtype=float)

submission_data_y = []
y = []

for i in range (0,8):
    for block in data:
        y.append(block[i])
    submission_data_y.append(y)
    y = []

In [5]:
# Join Data
X = {}
y = {}
for i in range(0,8): 
    X[i] = np.vstack([initial_data_X[i], submission_data_X[i]])
    y[i] = np.append(initial_data_y[i], submission_data_y[i])

In [6]:
# Save data
np.save("X.npy", X, allow_pickle=True)
np.save("y.npy", y, allow_pickle=True)